<a href="https://colab.research.google.com/github/janShi1105/science/blob/main/ML15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def convld(x, w, p=0, s=1):
  w_rot = np.array(w[::-1])
  x_padded = np.array(x)
  if p > 0:
    zero_pad = np.zeros(shape=p)
    x_padded = np.concatenate([zero_pad, x_padded, zero_pad])
  res = []
  for i in range(0, int((len(x_padded) - len(w_rot))/ s) + 1, s):
    res.append(np.sum(x_padded[i:i + w_rot.shape[0]] * w_rot))
  return np.array(res)

x = [1,3,2,4,5,6,1,3]
w = [1,0,3,1,2]
print('Convld Implementation:', convld(x, w, p=2, s=1))
print('Numpy Results: ', np.convolve(x,w, mode='same'))

Convld Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]
Numpy Results:  [ 5 14 16 26 24 34 19 22]


In [2]:
import numpy as np
import scipy.signal

def conv2d(X, W, p=(0,0), s=(1,1)):
  W_rot =  np.array(W)[::-1, ::-1]
  X_orig = np.array(X)
  n1 = X_orig.shape[0] + 2 * p[0]
  n2 = X_orig.shape[1] + 2* p[1]
  X_padded = np.zeros(shape=(n1,n2))
  X_padded[p[0]:p[0] + X_orig.shape[0], p[1]:p[1] + X_orig.shape[1]] = X_orig
  res = []
  for i in range(0, int((X_padded.shape[0] - W_rot.shape[0]) / s[0]) + 1, s[0]):
    res.append([])
    for j in range(0, int((X_padded.shape[1] - W_rot.shape[1]) /s[1]) + 1, s[1]):
      X_sub = X_padded[i: i + W_rot.shape[0], j:j+W_rot.shape[1]]
      res[-1].append(np.sum(X_sub * W_rot))
  return (np.array(res))

X = [[1,3,2,4], [5,6,1,3], [1,2,0,2], [3,4,3,2]]
W = [[1,0,3], [1,2,1], [0,1,1]]
print('Conv2d Implementation:\n', conv2d(X, W, p=(1,1), s=(1,1)))

print('Scipy Results: \n', scipy.signal.convolve2d(X, W, mode='same'))

Conv2d Implementation:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]
Scipy Results: 
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
img_raw = tf.io.read_file('drive/MyDrive/example-image.png')
img = tf.image.decode_image(img_raw)
print('Image shape: ', img.shape)

Image shape:  (252, 221, 3)


In [5]:
!ls

drive  sample_data


In [6]:
!pip install imageio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import imageio
img = imageio.imread('drive/MyDrive/example-image.png')
print('Image shape: ', img.shape)
print('Number of channels: ', img.shape[2])
print('Image data type: ', img.dtype)

Image shape:  (252, 221, 3)
Number of channels:  3
Image data type:  uint8


In [8]:
print(img[100:102, 100:102, :])

[[[179 134 110]
  [182 136 112]]

 [[180 135 111]
  [182 137 113]]]


In [9]:
from tensorflow import keras 
comv_layer = keras.layers.Conv2D(filters=16, kernel_size=(3,3), kernel_regularizer=keras.regularizers.l2(0.001))
fc_layer = keras.layers.Dense(units=16, kernel_regularizer=keras.regularizers.l2(0.001))

In [12]:
bce_probas = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce_logits = tf.keras.losses.BinaryCrossentropy(from_logits=True)

logits = tf.constant([0.8])
probas = tf.keras.activations.sigmoid(logits)
tf.print('BCE (w probas): {:.4f}'.format(bce_probas(y_true=[1], y_pred=probas)), '(w Logits): {:.4f}'.format(bce_logits(y_true=[1], y_pred=logits)))

BCE (w probas): 0.3711 (w Logits): 0.3711


In [14]:
cce_probas = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
cce_logits = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
logits = tf.constant([[1.5, 0.8, 2.1]])
probas = tf.keras.activations.softmax(logits)
tf.print(
    'CCE (w probas): {:.4f}'.format(cce_probas(y_true=[[0,0,1]], y_pred=probas)),
    '(w logits): {:.4f}'.format(cce_logits(y_true=[[0,0,1]], y_pred=logits))
)                                                    

CCE (w probas): 0.5996 (w logits): 0.5996


In [15]:
sp_cce_probas = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
sp_cce_logits = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

tf.print(
    'Sparse CCE (w Probas): {:.4f}'.format(sp_cce_probas(y_true=[2], y_pred=probas)),
    '(w logits): {:.4f}'.format(sp_cce_logits(y_true=[2], y_pred=logits))
)

Sparse CCE (w Probas): 0.5996 (w logits): 0.5996


In [19]:
import tensorflow_datasets as tfds
mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
datasets = mnist_bldr.as_dataset(shuffle_files=False)
mnist_train_orig = datasets['train']
mnist_test_orig = datasets['test']

In [22]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20
mnist_train = mnist_train_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32)/ 255.0, tf.cast(item['label'], tf.int32))
)
mnist_test = mnist_test_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32)/ 255.0, tf.cast(item['label'], tf.int32))
)

In [23]:
tf.random.set_seed(1)
mnist_train = mnist_train.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=False)
mnist_valid = mnist_train.take(10000).batch(BATCH_SIZE)
mnist_train = mnist_train.skip(10000).batch(BATCH_SIZE)